In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [2]:
from keras.initializers import TruncatedNormal

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,GRU
from keras.layers import Lambda
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import RMSprop,adam
from keras.callbacks import History
from keras.layers import Input, Dense, Masking, Dropout
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import TensorBoard
from keras.layers.normalization import BatchNormalization
from keras import layers

Using TensorFlow backend.
C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\podkd\.conda\envs\park\lib\site-packages\tensorflo

In [4]:
from docker.src.data_loader import DataLoader

# from docker.src.model import DataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.metrics import f1_score, roc_auc_score

# datagen

In [2]:
# measur = pd.read_csv("./data/train/MEASUREMENT_NICU.csv", encoding='windows-1252')
# condi = pd.read_csv("./data/train/CONDITION_OCCURRENCE_NICU.csv", encoding='windows-1252')
# outcome = pd.read_csv("./data/train/OUTCOME_COHORT.csv", encoding='windows-1252')
# person = pd.read_csv("./data/train/PERSON_NICU.csv", encoding='windows-1252')

# list1 =np.zeros([10,2])

# list1[:,0]=range(10,20)
# list1[:,1]=100000

# for i,n in list1:
#     measure_copy= measur[:int(n)].copy()
#     measure_copy.PERSON_ID =int(i)
#     measur = pd.concat([measur,measure_copy],axis=0)

# measur.to_csv("./MEASUREMENT_NICU.csv", encoding='windows-1252')

# list1 =np.zeros([10,2])

# list1[:,0]=range(10,20)
# list1[:,1]=100000

# condi_copy= condi.copy()

# for i in range(10,20):
#     condi_copy2= condi.copy()
#     condi_copy2.PERSON_ID =int(i)
#     condi_copy = pd.concat([condi_copy,condi_copy2],axis=0)
    

# condi_copy.to_csv("./CONDITION_OCCURRENCE_NICU.csv", encoding='windows-1252')

# import random
# np.random.choice(2,1,p=[0.8,0.2])[0]

# for i in range(10,20):
#     outcome_copy = outcome[:100].copy()
#     outcome_copy.SUBJECT_ID =int(i)
#     if np.random.choice(2,1,p=[0.6,0.4])[0] == 1:
#         print("1")
#         outcome_copy.LABEL = np.random.choice(2,100,p=[0.9,0.1])
#     outcome = pd.concat([outcome,outcome_copy],axis=0)

# outcome.to_csv("./OUTCOME_COHORT.csv", encoding='windows-1252')

# person_copy= person.copy()

# for i in range(10,20):
#     person_copy2 = person.copy()
#     person_copy2.PERSON_ID =int(i)
#     person_copy = pd.concat([person_copy,person_copy2],axis=0)

# person_copy.to_csv("./PERSON_NICU.csv", encoding='windows-1252')

In [5]:
data_path = './data'
task_id = 'local_test'
task_path = os.path.join(data_path, 'volume', task_id)
log_path = os.path.join(data_path, 'volume', 'logs')
task_log_path = os.path.join(log_path, task_id)
if not os.path.exists(task_path):
  os.mkdir(task_path)
if not os.path.exists(log_path):
  os.mkdir(log_path)
if not os.path.exists(task_log_path):
  os.mkdir(task_log_path)


In [6]:
task_path

'./data\\volume\\local_test'

In [7]:
import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_loader, fraction):
        'Initialization'
        self.xt, self.yt,self.nx, self.ny = data_loader()
        self.fraction = fraction
        
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.nx) / 5))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
            #positive_valid patient중 negative data undersampling    
        rand_false2 = np.random.choice(self.nx.shape[0], size=int(np.floor(self.nx.shape[0]*self.fraction)))
        random_nx = self.nx[rand_false2]
        random_ny = self.ny[rand_false2]
        
        train_x = np.concatenate([self.xt,random_nx], axis=0)
        train_y = np.concatenate([self.yt,random_ny], axis=0)
            
        if len(train_x) == len(train_y):
            p = np.random.permutation(len(train_x))
            train_x = train_x[p]
            self.train_y = train_y[p]  
        else:
            print("there is non match")
        self.train_x = pad_sequences(train_x, padding='post', value=-5)
        return (self.train_x, self.train_y)   
    
    def shape(self):
        return self.train_x.shape

In [9]:
data_loader = DataLoader(data_path=os.path.join(data_path, 'train'),
                         common_path=os.path.join(data_path, 'volume'),
                         task_path=task_path,
                        is_train=True)


data_loader extract_outcome_cohort time: 0.013001441955566406
data_loader extract_person time: 0.010003805160522461
data_loader extract_condition time: 0.007995128631591797
data_loader extract_measurement time: 2.696000337600708
data_loader groupby_hour_condition time: 0.010998010635375977
there is Na after interpolation
data_loader groupby_hour_measurement time: 9.663999795913696
data_loader make_person_sequence time: 0.1340022087097168
X (1592,)
Y (1592,)
Key (1592, 2)
data_loader make_data time: 0.28100061416625977
on_split
data_loader split_data time: 0.04399514198303223


In [10]:
a,b,c,d = data_loader.get_train_data()
x,y,z,t = data_loader.get_valid_data()

# data_loader = DataLoader(data_path=os.path.join(data_path, 'train'),
#                          common_path=os.path.join(data_path, 'volume'),
#                          task_path=task_path)
train_datagen = DataGenerator(data_loader.get_train_data,fraction = 0.2)
valid_datagen = DataGenerator(data_loader.get_valid_data,fraction = 0.2)

In [11]:
help(Model.fit_generator)

Help on function fit_generator in module keras.engine.training:

fit_generator(self, generator, steps_per_epoch=None, epochs=1, verbose=1, callbacks=None, validation_data=None, validation_steps=None, validation_freq=1, class_weight=None, max_queue_size=10, workers=1, use_multiprocessing=False, shuffle=True, initial_epoch=0)
    Trains the model on data generated batch-by-batch by a Python generator
    (or an instance of `Sequence`).
    
    The generator is run in parallel to the model, for efficiency.
    For instance, this allows you to do real-time data augmentation
    on images on CPU in parallel to training your model on GPU.
    
    The use of `keras.utils.Sequence` guarantees the ordering
    and guarantees the single use of every input per epoch when
    using `use_multiprocessing=True`.
    
    # Arguments
        generator: A generator or an instance of `Sequence`
            (`keras.utils.Sequence`) object in order to avoid
            duplicate data when using multipro

In [15]:
model_input = Input((None, 122))
x = model_input

x = Masking(mask_value=-5)(x)
x = Dense(64, activation = 'tanh',name = '1stDense')(x)
rnn_layers = [32]
for idx, node in enumerate(rnn_layers):
  return_sequences = False if idx == len(rnn_layers) - 1 else True

  x = GRU(node,activation = 'relu',
          return_sequences=return_sequences)(x)
  x = BatchNormalization()(x)
  x = Dropout(0.5)(x)

model_output = Dense(1, activation = 'sigmoid', kernel_initializer=TruncatedNormal(stddev=0.01))(x)

loss = 'binary_crossentropy'

model = Model(model_input, model_output)
model.compile(optimizer=adam(lr =0.001), loss=loss, metrics=['accuracy'])


In [20]:
callbacks = [
    ModelCheckpoint(filepath=os.path.join(task_path, 'model-{epoch:02d}-{val_loss:2f}.hdf5'),
                    monitor='val_loss',
                    mode='min',
                    save_best_only=True,
                    save_weights_only=False,
                    verbose=True
    ),
    TensorBoard(log_dir=task_log_path,
                write_graph=True
    ),
     EarlyStopping(monitor='val_loss', min_delta=0, patience=50, verbose=2, mode='auto')

]
    
model = SimpleRNNModel(shape=122)

model.train(train_datagen, valid_datagen, epochs=200, valid_steps = 10, 
            step_epoch = 10, verbose=0, callbacks=callbacks,workers=-1)



Epoch 00001: val_loss improved from inf to 0.69987, saving model to ./data\volume\local_test\model-01-0.699868.hdf5

Epoch 00002: val_loss improved from 0.69987 to 0.69887, saving model to ./data\volume\local_test\model-02-0.698872.hdf5

Epoch 00003: val_loss improved from 0.69887 to 0.69839, saving model to ./data\volume\local_test\model-03-0.698387.hdf5

Epoch 00004: val_loss improved from 0.69839 to 0.68854, saving model to ./data\volume\local_test\model-04-0.688543.hdf5

Epoch 00005: val_loss improved from 0.68854 to 0.67857, saving model to ./data\volume\local_test\model-05-0.678566.hdf5

Epoch 00006: val_loss did not improve from 0.67857

Epoch 00007: val_loss improved from 0.67857 to 0.66291, saving model to ./data\volume\local_test\model-07-0.662913.hdf5

Epoch 00008: val_loss improved from 0.66291 to 0.65606, saving model to ./data\volume\local_test\model-08-0.656060.hdf5

Epoch 00009: val_loss improved from 0.65606 to 0.63676, saving model to ./data\volume\local_test\model-0

KeyboardInterrupt: 

In [55]:
a1,a2 = train_datagen.__getitem__(1)

train_datagen.__len__()

a1.shape

## model prediction (inference)

In [12]:
from docker.src.data_loader import DataLoader
data_loader = DataLoader(data_path=os.path.join(data_path, 'train'),
                         common_path=os.path.join(data_path, 'volume'),
                         task_path=task_path,
                        is_train=False)


data_loader extract_outcome_cohort time: 0.0110015869140625
data_loader extract_person time: 0.009998559951782227
data_loader extract_condition time: 0.007002830505371094
data_loader extract_measurement time: 2.248997688293457
data_loader groupby_hour_condition time: 0.016000032424926758
there is Na after interpolation
data_loader groupby_hour_measurement time: 7.6969990730285645
data_loader make_person_sequence time: 0.07199954986572266
X (1592,)
Key (1592, 2)
data_loader make_data time: 0.19500136375427246
not_in_split
data_loader split_data time: 0.06700372695922852


In [13]:
from docker.src.model import SimpleRNNModel
from tensorflow.keras.models import load_model

model = SimpleRNNModel(shape=122)

# 모델 로드
model2=model.load(task_path)

In [14]:
test_x = data_loader.get_infer_data()
test_x.shape[2]

122

In [16]:
y_key = data_loader.key
y_pred = model.predict(test_x)
y_pred = pd.DataFrame(y_pred, columns=['pred'])


In [17]:
y_pred = pd.concat([y_key, y_pred], axis=1)

out_key = data_loader.cohort_df[['SUBJECT_ID', 'COHORT_END_DATE']]
merged_df = pd.merge(out_key, y_pred, on=['SUBJECT_ID', 'COHORT_END_DATE'], how='left')

merged_df['LABEL_PROBABILITY'] = merged_df.pred
merged_df['LABEL'] = (merged_df['LABEL_PROBABILITY'] > thr).astype(np.int32)

merged_df[['LABEL', 'LABEL_PROBABILITY']].to_csv(os.path.join(data_path, 'output', 'output.csv'), index=False)
